In [7]:
import os
import dotenv
import boto3
import json

# AWS 계정 자격 증명 및 리전 설정
env_path = dotenv.find_dotenv()
dotenv.load_dotenv(env_path)

aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
region_name = 'ap-northeast-2'

s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

def load_data(bucket_name, object_key):

    # JSON 파일 읽어오기
    try:
        response = s3.get_object(Bucket=bucket_name, Key=object_key)
        json_data = json.loads(response['Body'].read().decode('utf-8'))
        return json_data
    
    except Exception as e:
        print(e)

In [8]:
json_data = load_data('mathcat-bucket', 'irt_result/yyyy=2024/mm=03/dd=13/irt_result.json')


In [9]:
import numpy as np

json_data = load_data('mathcat-bucket', 'irt_result/yyyy=2024/mm=03/dd=13/irt_result.json')

def json_to_array(json_data):

    result_array = []

    for i in range(len(json_data['disc'])):
        new_array = [json_data['disc'][i], json_data['diff'][i], 0, 1]
        result_array.append(new_array)

    return result_array

result_array = json_to_array(json_data)
np.array(result_array)

array([[ 1.24731219,  0.49697632,  0.        ,  1.        ],
       [ 1.2732867 , -0.57974267,  0.        ,  1.        ],
       [ 1.07688701,  0.56360698,  0.        ,  1.        ],
       ...,
       [ 0.75963587, -0.41184527,  0.        ,  1.        ],
       [ 0.08181728,  0.06815594,  0.        ,  1.        ],
       [ 0.82494938, -0.61504519,  0.        ,  1.        ]])

In [10]:
# 초기 4문제 받는 부분

# 난이도 추출
diff = np.array(result_array)[:, 1]

# 중앙값 계산
median_value = np.median(diff)

# 중앙값과 가장 가까운 네 개의 행의 인덱스를 찾기 위해 argsort 사용
# 두 번째 열의 값과 중앙값의 차이를 기준으로 오름차순 정렬 후 인덱스 반환
sorted_indices = np.argsort(np.abs(diff - median_value))

# 가장 작은 네 개의 인덱스 추출
selected_indices = sorted_indices[:4]

# 리스트 초기화
initial_item_ids = []

for index in selected_indices:
    item_id = json_data['item_ids'][str(index)]
    initial_item_ids.append(item_id)

initial_item_ids

['quiz30062306', 'quiz30066214', 'quiz30074073', 'quiz30065549']

In [11]:
from catsim.initialization import *
from catsim.selection import *
from catsim.estimation import *
from catsim.stopping import *

# 초기화
initializer = FixedPointInitializer(0)
selector = UrrySelector()
estimator = NumericalSearchEstimator()
stopper = MinErrorStopper(0.6)

In [14]:
from catsim.simulation import *
import numpy as np

administered_items = []
responses = []
index = 0

class Simulator:
    def __init__(self, result_array, init_items, initializer, selector, estimator, stopper):
        self.result_array = result_array
        self.init_items = init_items
        self.initializer = initializer
        self.selector = selector
        self.estimator = estimator
        self.stopper = stopper

    def recommend_next(self, quiz=False):
        global index
        global est_theta
        global administered_items

        ## 다음 문제 추천
        # 초기 4문제는 리스트에서 반환
        if len(administered_items) < self.init_items:

            if quiz == True:
                return initial_item_ids[index]
            
            item_index = list(json_data['item_ids'].values()).index(initial_item_ids[index])
            administered_items.append(item_index)

            # 문제에 대한 응답 추가
            correct = input(f"Did the user answer the item {initial_item_ids[index]} correctly? (True/False): ").lower()
            
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
            
            responses.append(correct == 'true')
            print(f'Did the user answer the item {initial_item_ids[index]} correctly?: {responses[index]}')

            
        
        # 5번째 문제부터는 사용자의 학습 수준에 맞게 추천
        else:
            
            item_index = self.selector.select(items=np.array(self.result_array), administered_items=administered_items, est_theta=est_theta)

            if quiz == True:
                return json_data['item_ids'][str(item_index)]
            
            administered_items.append(item_index)

            # 문제에 대한 응답 추가
            correct = input(f"Did the user answer the item {json_data['item_ids'][str(item_index)]} correctly? (True/False): ").lower()

            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")

            responses.append(correct == 'true')
            print(f"Did the user answer the item {json_data['item_ids'][str(item_index)]} correctly?: {responses[index]}")


        # 사용자의 학습 수준 추정
            
        # if index == 0:
        #     est_theta = self.initializer.initialize()
        #     est_theta = self.estimator.estimate(
        #         items=np.array(self.result_array),
        #         administered_items=administered_items,
        #         response_vector=responses,
        #         est_theta=est_theta
        #     )
            
        # else:
        #     est_theta = self.estimator.estimate(
        #         items=np.array(self.result_array),
        #         administered_items=administered_items,
        #         response_vector=responses,
        #         est_theta=est_theta
        #     )
            
        if index == 3:
            est_theta = self.initializer.initialize()
            est_theta = self.estimator.estimate(
                items=np.array(self.result_array),
                administered_items=administered_items,
                response_vector=responses,
                est_theta=est_theta
            )
            
        elif index > 3:
            est_theta = self.estimator.estimate(
                items=np.array(self.result_array),
                administered_items=administered_items,
                response_vector=responses,
                est_theta=est_theta
            )
        
        else:
            est_theta = 0
        
        if self.stopper.stop(administered_items=np.array(self.result_array)[administered_items], theta=est_theta):
                
            # 사용자의 학습 수준 출력
            print("시험 종료")
            print("Final estimated proficiency:", est_theta)
            
            return est_theta, administered_items, responses, index, True

        index = index + 1

        return est_theta, administered_items, responses, index, False

In [15]:
# Simulator 객체 생성
simulator = Simulator(result_array, 4, initializer, selector, estimator, stopper)


1. 

In [16]:
simulator.recommend_next(quiz=True)

'quiz30062306'

In [17]:
simulator.recommend_next()

Did the user answer the item quiz30062306 correctly?: True


(0, [1202], [True], 1, False)

2.

In [18]:
simulator.recommend_next(quiz=True)

'quiz30066214'

In [19]:
simulator.recommend_next()

Did the user answer the item quiz30066214 correctly?: True


(0, [1202, 1055], [True, True], 2, False)

3.

In [20]:
simulator.recommend_next(quiz=True)

'quiz30074073'

In [21]:
simulator.recommend_next()

Did the user answer the item quiz30074073 correctly?: True


(0, [1202, 1055, 968], [True, True, True], 3, False)

4.

In [22]:
simulator.recommend_next(quiz=True)

'quiz30065549'

In [23]:
simulator.recommend_next()

Did the user answer the item quiz30065549 correctly?: True


(1.6665791273117065,
 [1202, 1055, 968, 946],
 [True, True, True, True],
 4,
 False)

5.

In [24]:
simulator.recommend_next(quiz=True)

'quiz30059043'

In [25]:
simulator.recommend_next()

Did the user answer the item quiz30059043 correctly?: True


(2.49986869096756,
 [1202, 1055, 968, 946, 308],
 [True, True, True, True, True],
 5,
 False)

6.

In [26]:
simulator.recommend_next(quiz=True)

'quiz30051310'

In [27]:
simulator.recommend_next()

Did the user answer the item quiz30051310 correctly?: True


(2.9165134727954865,
 [1202, 1055, 968, 946, 308, 241],
 [True, True, True, True, True, True],
 6,
 False)

7.

In [28]:
simulator.recommend_next(quiz=True)

'quiz30055590'

In [29]:
simulator.recommend_next()

Did the user answer the item quiz30055590 correctly?: True


(3.1248358637094498,
 [1202, 1055, 968, 946, 308, 241, 641],
 [True, True, True, True, True, True, True],
 7,
 False)

8.

In [30]:
simulator.recommend_next(quiz=True)

'quiz30046376'

In [31]:
simulator.recommend_next()

Did the user answer the item quiz30046376 correctly?: True


(3.2289970591664314,
 [1202, 1055, 968, 946, 308, 241, 641, 87],
 [True, True, True, True, True, True, True, True],
 8,
 False)

9.

In [32]:
simulator.recommend_next(quiz=True)

'quiz30060033'

In [33]:
simulator.recommend_next()

Did the user answer the item quiz30060033 correctly?: True


(3.2810776568949223,
 [1202, 1055, 968, 946, 308, 241, 641, 87, 52],
 [True, True, True, True, True, True, True, True, True],
 9,
 False)

10.

In [34]:
simulator.recommend_next(quiz=True)

'quiz30051309'

In [35]:
simulator.recommend_next()

Did the user answer the item quiz30051309 correctly?: True


(3.3071179557591677,
 [1202, 1055, 968, 946, 308, 241, 641, 87, 52, 196],
 [True, True, True, True, True, True, True, True, True, True],
 10,
 False)

11.

In [36]:
simulator.recommend_next(quiz=True)

'quiz30061291'

In [37]:
simulator.recommend_next()

Did the user answer the item quiz30061291 correctly?: True


(3.3201381051912904,
 [1202, 1055, 968, 946, 308, 241, 641, 87, 52, 196, 398],
 [True, True, True, True, True, True, True, True, True, True, True],
 11,
 False)

12.

In [38]:
simulator.recommend_next(quiz=True)

'quiz30057006'

In [39]:
simulator.recommend_next()

Did the user answer the item quiz30057006 correctly?: True
시험 종료
Final estimated proficiency: 3.3266481799073517


(3.3266481799073517,
 [1202, 1055, 968, 946, 308, 241, 641, 87, 52, 196, 398, 1260],
 [True, True, True, True, True, True, True, True, True, True, True, True],
 11,
 True)

In [6]:
from catsim.simulation import *
from catsim.initialization import *
from catsim.selection import *
from catsim.estimation import *
from catsim.stopping import *
from catsim.irt import icc
import numpy as np
import matplotlib.pyplot as plt


result_array = []

for i in range(len(json_data['disc'])):
    new_array = [json_data['disc'][i], json_data['diff'][i], 0, 1]
    result_array.append(new_array)

np.array(result_array)
## ---------------------------------------- 여기까지 json 열고 result_array 댓고오는거임 ----------------------------------------------------

class Simulator:
    def __init__(self, result_array, max_items, initializer, selector, estimator, stopper):
        #simulator로 돌려야 되니까 self 로 그냥 다 받았습니다. 
        self.result_array = result_array
        self.max_items = max_items
        self.initializer = initializer
        self.selector = selector
        self.estimator = estimator
        self.stopper = stopper

    def simulate(self, verbose=False):
        global initial_item_ids
        administered_items = []
        responses = []
        
        # 초기 4문제의 인덱스를 찾아서 administered_items에 추가하고 응답 벡터에 대응하는 응답을 추가
        for item_id in initial_item_ids:
            item_index = list(json_data['item_ids'].values()).index(item_id)
            administered_items.append(item_index)

            # 초기 문제에 대한 응답 추가
            correct = input(f"Did the user answer the item {item_id} correctly? (True/False): ").lower()
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
                continue
            responses.append(correct == 'true')

        # 초기화 단계
        est_theta = self.initializer.initialize()

        est_theta = self.estimator.estimate(
        items=np.array(self.result_array),
        administered_items=administered_items,
        response_vector=responses,
        est_theta=est_theta
    )
        # 여기까지가, 이제 해당 4문제들을 난이도가 중간값에 근사한 걸로 넣어가지고 진행했구요
        # 여기서 이제 사용자가 입력받아 True or False로 알겠지

    # 초기 4문제 후의 능력치 출력
        if verbose:
            print("Initial 4 items answered. Estimated proficiency:", est_theta)
            ## 여기까지 이제 4문제를 풀고 난후의 theta 값 즉, 사용자의 능력치를 추정

        while True:
            # 추천 단계
            if len(administered_items) >= len(initial_item_ids):
                item_index = self.selector.select(items=np.array(self.result_array), administered_items=administered_items, est_theta=est_theta)
            else:
                # 초기 문제를 추천하지 않도록 건너뛰기
                item_index = administered_items[len(administered_items)]

            next_item_id = json_data['item_ids'][str(item_index)]
            ## 여기까지 이제 문제를 select해서 퀴즈코드 추천  -> 문제 추천해줘

            # 평가 단계
            true_theta = 0.0
            a, b, c, d = np.array(self.result_array)[item_index]
            prob = icc(true_theta, a, b, c, d)
            correct = input(f"Did the user answer the item {next_item_id} correctly? (True/False): ").lower()
            if correct not in ['true', 'false']:
                print("Invalid input. Please enter either 'True' or 'False'.")
                continue
            correct = (correct == 'true')

            # 여기까지가 그 문제를 맞추고, 틀렸는지 이제 estimation 해줘요

            # 중단 단계
            if len(administered_items) >= self.max_items or self.stopper.stop(administered_items=np.array(self.result_array)[administered_items], theta=est_theta):
                break
            #max_item 은 필요없긴 한데, self stopper에서 종단기준이 만족하면 stop

            # 결과 기록
            administered_items.append(item_index)
            responses.append(correct)

            if verbose:
                print('Estimated proficiency, given answered items:', est_theta)
                print('Next item to be administered:', next_item_id)
                print('Probability to correctly answer item:', prob)
                print('Did the user answer the selected item correctly?', correct)
                print("responses: ", responses)

            # 능력치 업데이트
            est_theta = self.estimator.estimate(
                items=np.array(self.result_array),
                administered_items=administered_items,
                response_vector=responses,
                est_theta=est_theta
            )

        if verbose:
            print("responses: ", responses)
        print("사용자의 최종 능력치 (Estimated Proficiency):", est_theta)
        # 사용자의 최종 능력치를 추정해줌.


## --------------------- 여기까지가 클래스 정의고 밑에 클래스 돌리는거 ---------------------------------
initializer = FixedPointInitializer(0)
selector = UrrySelector()
estimator = NumericalSearchEstimator()
stopper = MinErrorStopper(.6)

s = Simulator(np.array(result_array), 20, initializer, selector, estimator, stopper)
s.simulate(verbose=True)


# ----- 분기 처리 해드리고 싶은데 그냥 알아서 해주세요 ㅅㄱ----------------------

Initial 4 items answered. Estimated proficiency: 1.6665791273117065
Estimated proficiency, given answered items: 1.6665791273117065
Next item to be administered: quiz30059043
Probability to correctly answer item: 0.10365764256641782
Did the user answer the selected item correctly? True
responses:  [True, True, True, True, True]
Estimated proficiency, given answered items: 2.49986869096756
Next item to be administered: quiz30051310
Probability to correctly answer item: 0.038517233065577396
Did the user answer the selected item correctly? True
responses:  [True, True, True, True, True, True]
Estimated proficiency, given answered items: 2.9165134727954865
Next item to be administered: quiz30055590
Probability to correctly answer item: 0.06318217660694031
Did the user answer the selected item correctly? True
responses:  [True, True, True, True, True, True, True]
Estimated proficiency, given answered items: 3.1248358637094498
Next item to be administered: quiz30046376
Probability to correct